In [25]:
from typing import Literal
import numpy as np

In [26]:
class GolayCode:
    """Реализация двоичного кода Голея G24"""
    
    def __init__(self):
        # Порождающая матрица G24 в систематической форме [I12 | A]
        # A построена на основе квадратичных вычетов по модулю 11
        self.I12 = np.eye(12, dtype=int)
        
        # Матрица A (12x12) для G24
        # Первые 11 строк - циклические сдвиги последовательности квадратичных вычетов
        # Последняя строка - все 1, кроме последнего бита 0
        quadratic_residues = [0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0]  # для i=0..10
        
        A = np.zeros((12, 12), dtype=int)
        
        # Заполняем первые 11 строк циклическими сдвигами
        for i in range(11):
            shift = quadratic_residues[-i:] + quadratic_residues[:-i]  # циклический сдвиг
            A[i, :11] = shift
            A[i, 11] = sum(shift) % 2  # бит четности
        
        # Последняя строка матрицы A
        A[11, :11] = 1
        A[11, 11] = 0
        
        self.A = A
        self.G24 = np.concatenate((self.I12, self.A), axis=1)
        
        # Проверочная матрица H24 = [A^T | I12] (так как код самодуальный)
        self.H24 = np.concatenate((A.T, self.I12), axis=1)
        
        # Таблица синдромов для быстрого декодирования (веса ошибок 0-3)
        self.syndrome_table = {}
        self._build_syndrome_table()
    
    def _build_syndrome_table(self):
        """Построение таблицы синдромов для ошибок веса <= 3"""
        # Генерируем все ошибки веса 0-3 в 24-битных словах
        from itertools import combinations
        
        positions = list(range(24))
        
        # Вес 0 (нет ошибок)
        error = np.zeros(24, dtype=int)
        syndrome = self._compute_syndrome(error)
        self.syndrome_table[tuple(syndrome)] = error
        
        # Вес 1
        for pos in positions:
            error = np.zeros(24, dtype=int)
            error[pos] = 1
            syndrome = self._compute_syndrome(error)
            self.syndrome_table[tuple(syndrome)] = error
        
        # Вес 2
        for pos1, pos2 in combinations(positions, 2):
            error = np.zeros(24, dtype=int)
            error[pos1] = 1
            error[pos2] = 1
            syndrome = self._compute_syndrome(error)
            self.syndrome_table[tuple(syndrome)] = error
        
        # Вес 3
        for pos1, pos2, pos3 in combinations(positions, 3):
            error = np.zeros(24, dtype=int)
            error[pos1] = 1
            error[pos2] = 1
            error[pos3] = 1
            syndrome = self._compute_syndrome(error)
            self.syndrome_table[tuple(syndrome)] = error
    
    def _compute_syndrome(self, vector):
        """Вычисление синдрома для вектора длины 24"""
        return np.dot(vector, self.H24.T) % 2
    
    def encode(self, message: np.ndarray):
        """Кодирование 12-битного сообщения в 24-битное слово G24"""
        if len(message) != 12:
            raise ValueError("Сообщение должно быть длиной 12 бит")
        
        codeword = np.dot(message, self.G24) % 2
        return codeword
    

    def decode(self, received):
        """Декодирование 24-битного вектора в сообщение G24"""
        received = np.array(received, dtype=int)
        
        # Вычисляем синдром
        syndrome = self._compute_syndrome(received)
        syndrome_tuple = tuple(syndrome)
        
        # Ищем синдром в таблице
        if syndrome_tuple in self.syndrome_table:
            error = self.syndrome_table[syndrome_tuple]
            corrected = (received + error) % 2
            
            # Извлекаем информационные биты (первые 12)
            message = corrected[:12]
            return message, error
        else:
            # Ошибка веса > 3, не исправляется гарантированно
            raise ValueError("Ошибка веса > 3, невозможно исправить")
    
    def add_errors(self, codeword, num_errors, positions=None):
        """Добавление ошибок в кодовое слово"""
        codeword = np.array(codeword, dtype=int).copy()
        
        if positions is None:
            # Выбираем случайные позиции для ошибок
            positions = np.random.choice(len(codeword), num_errors, replace=False)
        
        for pos in positions:
            codeword[pos] = (codeword[pos] + 1) % 2  # Инвертируем бит
        
        return codeword, positions



In [ ]:
def demostrate(message: list[Literal[0,1]]):
    message = np.array(message, dtype=int)
    golay = GolayCode()

    print("=== КОД ГОЛЕЯ G24 ===\n")

    # Пример сообщения (12 бит)
    print(f"Исходное сообщение: {message}")

    # Кодируем в G24
    codeword_24 = golay.encode(message)
    print(f"Кодовое слово G24 (24 бита): {codeword_24}")
    print(f"Вес кодового слова: {sum(codeword_24)}")

    # Тест коррекции ошибок для G24
    print("--- Тест коррекции ошибок (G24) ---")

    # Добавляем 3 ошибки (максимально исправляемое количество)
    corrupted, error_pos = golay.add_errors(codeword_24, 3)
    print(f"Добавлены ошибки в позициях: {error_pos}")
    print(f"Принятый вектор с ошибками: {corrupted}")

    # Пытаемся исправить
    decoded_message, estimated_error = golay.decode(corrupted)
    print(f"Исправленная ошибка (вектор): {estimated_error}")
    print(f"Исправленное сообщение: {decoded_message}")
    print(f"Декодирование успешно: {all(decoded_message == message)}")



In [28]:
message = [1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0]
demostrate(message)

=== КОД ГОЛЕЯ G24 ===

Исходное сообщение: [1 0 0 1 1 0 1 1 0 0 1 0]
Кодовое слово G24 (24 бита): [1 0 0 1 1 0 1 1 0 0 1 0 1 1 0 0 0 1 1 1 0 1 0 0]
Вес кодового слова: 12
--- Тест коррекции ошибок (G24) ---
Добавлены ошибки в позициях: [23  1 15]
Принятый вектор с ошибками: [1 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 0 1 1 1 0 1 0 1]
Исправленная ошибка (вектор): [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]
Исправленное сообщение: [1 0 0 1 1 0 1 1 0 0 1 0]
Декодирование успешно: True
